In [ ]:
#pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
# function to check if the button is on the page
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# import the webdriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)


WebDriverException: ignored

In [ ]:
# Set the URL and options for the Chrome driver
url = 'https://www.yelp.com/search?find_desc=&find_loc=Rochester%2C+NY+14604'
# Initialize the DataFrame
df = pd.DataFrame()

# Loop through each page of the search results
for i in range(0, 20):
    # Construct the URL for the current page
    if i == 0:
        page_url = url
    else:
        page_url = f"{url}&start={i*10}"

    # Send a request to the URL and parse the HTML content with BeautifulSoup
    req = Request(page_url, headers={'User-Agent': 'Mozilla/6.0'})
    soup = BeautifulSoup(urlopen(req).read(),'html.parser')

    # Initialize the Chrome driver and wait for the page to load
    driver = webdriver.Chrome(options=options)
    driver.get(page_url)
    time.sleep(3)

    # Find all the restaurants on the page and extract their information
    restaurants = soup.find_all("li",class_="border-color--default__09f24__NPAKY")
    for restaurant in restaurants:
        try:
            businessName = restaurant.find('a','css-1m051bw').get_text()
            stars = restaurant.find('div','five-stars--regular__09f24__DgBNj').get("aria-label")
            noOfReviewers = restaurant.find('span','css-chan6m').get_text()
            cuisines = restaurant.find('span','css-11bijt4')
            if cuisines is not None:
                cuisine = cuisines.get_text()
            locations = restaurant.find('p',"css-dzq7l1")
            if locations is not None:
                location_inc = locations.get_text()
                splt = location_inc.split("$")
                location = splt[-1]
            priceRange = restaurant.find('span','priceRange__09f24__mmOuH css-1s7bx9e').get_text()
            topReview = restaurant.find('p','css-16lklrv').get_text()

            resturantstats = [businessName, stars, noOfReviewers, cuisine, location, priceRange, topReview]
            tempdf = pd.DataFrame(resturantstats).transpose()
            df = pd.concat([df,tempdf],ignore_index=True)
        except:
            pass

    # Quit the Chrome driver
    driver.quit()

# Print the first 10 rows of the DataFrame
print(df.head(10))



In [ ]:
df.columns=["businessName","stars","noOfReviewers","cusine","location","priceRange","topReview"]
df = df.drop_duplicates(keep='first')

In [ ]:
for i in range(len(df)):
    if((df.noOfReviewers[i]=='Upper Mount Hope') or (df.noOfReviewers[i]=='Center City')or(df.noOfReviewers[i]=='East End')or(df.noOfReviewers[i]=='')):
       df.noOfReviewers[i]=10
df.noOfReviewers=df.noOfReviewers.astype(int)

In [ ]:
for i in range(len(df)):
    stars=(df.stars[i].split(" "))
    df.stars[i]=stars[0]

In [ ]:
df.stars=df.stars.astype(float)

In [ ]:
df.dtypes

In [ ]:

# Plot the bar plot
sns.barplot(y='noOfReviewers', x='stars', data=df)
plt.ylabel('Number of Reviewers')
plt.xlabel('Star Rating')
plt.title('Relationship between Number of Reviewers and Star Rating')
plt.show()


In [ ]:
#identify top 10 cuisines of ROC

# Count the number of occurrences of each cuisine
cuisine_counts = df['cusine'].value_counts().head(10)

# Plot the bar chart
plt.bar(cuisine_counts.index, cuisine_counts.values)
plt.xticks(rotation=90)
plt.xlabel('Cuisine')
plt.ylabel('Number of Restaurants')
plt.title('Top 10 Cuisines in Rochester, NY')
plt.show()

In [ ]:
# Filter the DataFrame to include only the top 10 cuisines
top_cuisines = df['cusine'].value_counts().head(10).index
df_top_cuisines = df[df['cusine'].isin(top_cuisines)]

# Plot the box plot
sns.boxplot(y='cusine', x='stars', data=df_top_cuisines)
plt.xticks(rotation=0)
plt.ylabel('Cuisine')
plt.xlabel('Star Rating')
plt.title('Distribution of Star Ratings by Cuisine')
plt.show()

In [ ]:
 df['priceRange'].unique()

In [ ]:
for i in range(len(df)):
    if(df.location[i]=='GelatoIce Cream & Frozen YogurtPatisserie/Cake Shop'):
        df.location[i]='Downtown'
    elif(df.location[i]=='Coffee & TeaUpper Mount Hope'):
        df.location[i]='Mount Hope'
    elif(df.location[i]=='Coffee & TeaBreakfast & BrunchUpper Mount Hope'):
        df.location[i]='Mount Hope'
    elif(df.location[i]=='Patisserie/Cake ShopGelatoIce Cream & Frozen Yogurt'):
        df.location[i]='Downtown'
    elif(df.location[i]=='ItalianWine BarsTapas/Small PlatesEast End'):
        df.location[i]='East End'
    elif(df.location[i]=='BarsChicken WingsPizzaEast End'):
        df.location[i]='East End'
    elif(df.location[i]=='Ice Cream & Frozen YogurtPatisserie/Cake ShopGelato'):
        df.location[i]='Downtown'
    elif(df.location[i]=='DelisSandwichesBreakfast & BrunchCenter City'):
        df.location[i]='Center City'
    elif(df.location[i]=='Ice Cream & Frozen YogurtGelatoPatisserie/Cake Shop'):
        df.location[i]='Downtown'
    elif(df.location[i]=='Themed CafesPet AdoptionEast End'):
        df.location[i]='East End'

In [ ]:
# create a count plot
sns.countplot(x='location', hue='priceRange', data=df)

# set the title and axis labels
plt.title('Number of restaurants by price range and location')
plt.xlabel('Location')
plt.ylabel('Number of restaurants')

# show the plot
plt.show()

In [ ]:
import gensim.downloader as api
wv = api.load("glove-wiki-gigaword-300")
model=wv

In [ ]:
model_yummy=model['yummy']

In [ ]:
keys =list(df.topReview)
keys

In [ ]:
model['yummy']

In [ ]:
model.most_similar("yummy")

In [ ]:
corpus=list(df.topReview)

In [ ]:
import nltk
from nltk.corpus import stopwords

# Download the stop words from the NLTK library
nltk.download('stopwords')

# Load the stop words
stop_words = set(stopwords.words('english'))

# Remove the stop words from the corpus
filtered_corpus = []
for doc in corpus:
    words = doc.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_corpus.append(' '.join(filtered_words))

# Print the filtered corpus
print(filtered_corpus)


In [ ]:
import nltk
import string
from nltk.corpus import stopwords


# Download the stop words from the NLTK library
nltk.download('stopwords')

# Load the stop words
stop_words = set(stopwords.words('english'))

# Remove stop words and special characters from the corpus
filtered_corpus = []
for doc in corpus:
    # Remove special characters and convert to lower case
    doc = doc.translate(str.maketrans('', '', string.punctuation)).lower()
    # Split the document into words
    words = doc.split()
    # Filter out the stop words and rejoin the words into sentences
    filtered_words = [word for word in words if word not in stop_words]
    filtered_corpus.append(' '.join(filtered_words))

# Print the filtered corpus
print(filtered_corpus)


In [ ]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in df.lower().split() if word not in stoplist]
         for df in filtered_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
print(processed_corpus)

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

In [ ]:
from collections import Counter

# Example array of arrays
#words = [['apple', 'banana', 'apple'], ['banana', 'cherry', 'date'], ['apple', 'date', 'date']]

# Flatten the array of arrays into a single list of words
flattened_corpus = [processed_corpus for sublist in processed_corpus for processed_corpus in sublist]

# Use Counter to count the frequency of each word
word_counts = Counter(flattened_corpus)

# Print the word counts
print(word_counts)


In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# Generate the word cloud
wordcloud = WordCloud(background_color='white').generate_from_frequencies(word_counts)

# Display the generated image:
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.manifold import TSNE


# Define a function to preprocess text data
def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text).lower()
    # Split text into individual words
    words = text.split()
    # Remove stop words
    # words = [word for word in words if word not in stop_words]
    # Join the words back into a single string and return
    return ' '.join(words)

# Preprocess the text data
df['processed_review'] = df['topReview'].apply(preprocess_text)

# Tokenize the text data and train a Doc2Vec model
tagged_docs = [TaggedDocument(words=doc.split(), tags=[i]) for i, doc in enumerate(df['processed_review'])]
doc2vec_model = Doc2Vec(tagged_docs, vector_size=100, window=5, min_count=2, workers=4, epochs=100)

# Get the vector representation for each document
doc_vectors = [doc2vec_model.infer_vector(doc.words) for doc in tagged_docs]

# Convert the vectors to a numpy array
doc_vectors = np.array(doc_vectors)

# Compute t-SNE on the document vectors
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(doc_vectors)

# Plot the t-SNE results
plt.scatter(tsne_results[:,0], tsne_results[:,1])
plt.show()

